# Genetic Algorithm Learning and the Cobweb Model

The Cobweb model is a simple model of a market were:
- Firms are price takers
- Price is determined by total quantity of supply
- Goods produced are identical across firms

The Cobweb model can be sloved unsing a Genetic algoritm as showed by [Arifovic (1992)](https://www.uh.edu/hobby/eitm/_docs/past-lectures/2014-Lectures/Sunny-Wong/Genetic-Algorithm-Learning-and-the-Cobweb-Model.pdf). In this notebook we replicate a similar analysis.



In [18]:
import numpy as np
import pandas as pd
from copy import deepcopy
from alive_progress import alive_bar
from plotnine import *

## Individuals

Defining individuals. Each individual will have:
- Genotype X (a string of binary digits)
- Phenotype q (a real number)
- Fitness function F

In [19]:
class Individual(object):

    def __init__(self):

        self.X = []
        self.q=  0
        self.F = 0

This function is used to decode the genotype and obtain the phenotype. In the example we use a 5-bit string and a threshold of 2.

In [20]:
def get_quantity(x, q_bar, n_bits):
    q = 0
    for bit in x:
        q = (q << 1) | bit
    return q*q_bar/2**n_bits
get_quantity([0,1,0,1,1],2, 5)

0.6875

## GA

In [21]:
class cobwebGA(object):

    def __init__(self,
                 n_individuals=10,
                 n_generations=2,
                 n_bits=3,
                 q_bar=100,
                 cr=0.90,
                 mr=0.05,
                 A=10,
                 B=1,
                 x=1,
                 y=1):

        self.__cr = cr
        self.__mr = mr
        self.__k  = None

        self.__A= A
        self.__B= B
        self.__x=x
        self.__y=y
        self.__q_bar=q_bar

        self.__n_individuals = n_individuals
        self.__n_generations = n_generations
        self.__n_bits = n_bits

        self.__pop = []


    # Fitness function (profit of single firm)
    def __profits(self, P, q):
        pi=P*q-self.__x*q-0.5*self.__y*self.__n_individuals*(q**2)
        return pi

    # Sampling strategy that allows for generating individuals with non-unique genotypes
    def __sampling(self):

        for _ in range(self.__n_individuals):

            s   = Individual()
            s.X = list(np.random.randint(0, 2, n_bits))

            self.__pop.append(s)

    #function to extract market prices
    def __get_prices(self):
        Q= np.sum([s.q for s in self.__pop])
        P=self.__A-self.__B*Q
        return P

    #genotype-phenotype decoder (as described above)
    def __get_quantity(self, x):
        q = 0
        for bit in x:
            q = (q << 1) | bit
        q=q*self.__q_bar/2**self.__n_bits
        return q

    # Tournament selection
    def __tournament_selection(self):

        # first random selection
        selected = np.random.choice(self.__pop)

        # random selection of the remaining k-1 individuals
        for s in np.random.choice(self.__pop, self.__k-1):
            # check if better (e.g. perform a tournament)
            if s.F > selected.F:
                selected = s
        return selected


    # One-point crossover of two parents to create two children
    def __one_point_crossover(self, p1, p2):

        # children are copies of parents by default
        c1, c2 = deepcopy(p1), deepcopy(p2)

        # check for recombination
        if np.random.rand() < self.__cr:
            # select a crossover point that is not on the end of the string
            pt = np.random.randint(1, len(p1.X)-2)
            # perform crossover
            c1.X = p1.X[:pt] + p2.X[pt:]
            c2.X = p2.X[:pt] + p1.X[pt:]

        return c1, c2

    #repair method return children only if they are better than parents, otherwise parents will replace children
    def __repair(self, p1, p2, c1, c2, prev_price):
        #quantities are computed for each offspring
        c1.q=self.__get_quantity(c1.X)
        c2.q=self.__get_quantity(c2.X)
        #quantities are used to obtain profits with prices identical to previous generation
        c1.F=self.__profits(prev_price, c1.q)
        c2.F=self.__profits(prev_price, c2.q)
        #extract the best two individuals inside a family
        fam=[p1, p2, c1, c2]
        fam.sort(key=lambda x: x.F, reverse=True)
        c1=fam[0]
        c2=fam[1]
        return c1, c2

    #function to obtain rational expectations quantity equilibrium (later graphed with red dashed line)
    def __get_q_star(self):
        q_star=(self.__A-self.__x)/(self.__B+self.__y)
        return q_star

    #function to obtain rational expectations prices equilibrium (later graphed with red dashed line)
    def __get_p_star(self):
        q_star=self.__get_q_star()
        p_star=self.__A-self.__B*q_star
        return p_star

    # Integer mutation. The selected bit is replaced with another bit value
    def __mutation(self, s):

        for i in range(len(s.X)):
            # check for a mutation
            if np.random.rand() < self.__mr:
                s.X[i] = 1-s.X[i]


    def run(self, k=2, repair=False):

        self.__selection = None
        self.__k = k
        self.__selection = self.__tournament_selection

        #populaiton initialization

        self.__sampling()

        # keep track of prices, quantity and generation for the market
        outputs = {
            'prices': [],
            'quantity': [],
            'gen': []
        }

        # keep track of prices, quantity and generation for all the individuals
        all_outputs = {
            'prices': [],
            'quantity': [],
            'profits':[],
            'gen': []
        }

        # enumerate generations
        with alive_bar(self.__n_generations, force_tty=True) as bar:
            for gen in range(self.__n_generations):

                gen_q=0

                # extract individuals quantity
                for s in self.__pop:
                    s.q = self.__get_quantity(s.X)
                    all_outputs['quantity'].append(s.q)
                    gen_q=s.q+gen_q

                #extract price for generation
                price=self.__get_prices()
                #saving results of market price for interpretation
                outputs['prices'].append(price)


                # extract individuals profits given the generation price
                for s in self.__pop:
                    s.F = self.__profits(price, s.q)
                    all_outputs['profits'].append(s.F)
                    all_outputs['prices'].append(price)
                    all_outputs['gen'].append(gen)

                #saving results of market outputs an n of generation for interpretation
                outputs['quantity'].append(gen_q)
                outputs['gen'].append(gen)


                # select parents
                selected = [self.__selection() for _ in range(self.__n_individuals)]

                # create the next populations
                children = []

                for i in range(0, self.__n_individuals, 2):

                    # Get selected parents in pairs
                    p1, p2 = selected[i], selected[i+1]

                    # crossover
                    c1, c2 = self.__one_point_crossover(p1, p2)

                    # mutation
                    self.__mutation(c1)
                    self.__mutation(c2)

                    #repair (if selected)
                    if repair:
                        c1, c2=self.__repair(p1, p2, c1, c2, price)

                    # store for next generation
                    children.append(c1)
                    children.append(c2)

                # replace population
                self.__pop = children[:]
                #counting for progress bar
                bar()
                outputs_df=pd.DataFrame.from_dict(outputs)
                all_outputs_df=pd.DataFrame.from_dict(all_outputs)

        return outputs_df, all_outputs_df

    #graphs
    def graphs(self, variable='prices'):
        if variable in ['prices', 'quantity']:
            star=0
            if variable=='prices':
                star=self.__get_p_star()
            if variable=='quantity':
                star=self.__get_q_star()
            ggplt=(ggplot(outputs_df)
                   + aes(x='gen', y=variable)
                   + geom_line()
                   + geom_hline(yintercept=star, linetype="dashed", color = "red")
                   + labs(x='Generation', y=variable.capitalize())
                   + theme_classic()
                   )
            return ggplt

        if variable=='SD':
            ggplt=(ggplot(outputs_df)
             + aes(x='quantity', y='prices', color='gen')
             + geom_point()
             + theme_classic()
             + labs(x='Quantity', y='Price')
             )
            return ggplt

        if variable=='SD_full':
            ggplt=(ggplot(all_outputs_df)
                   + aes(x='quantity', y='prices', color='gen')
                   + geom_point()
                   + theme_classic()
                   + labs(x='Quantity', y='Price')
                   )
            return ggplt

## Results

Stable Case

In [22]:
n_generations = 100 #number of generations
n_individuals = 100 #number of individuals

n_bits = 13 #lenght of chromosome
q_bar=5

c_r = 0.9 #crossover rate
m_r = 0.005 #mutation rate

#demand parameters
A=100
B=0.02
#cost function parameters
x=3
y=1

Checking if result correspond to rational expectations model

In [23]:
def get_q_star(A, B, x, y, n):
    q_star=(A-x)/(n*(B+y))
    return q_star

def get_p_star(A, B, x, y):
    p_star=A-B*((A-x)/((B+y)))
    return p_star

q_star=get_q_star(A=A, B=B, x=x, y=y, n=n_individuals)
p_star=get_p_star(A=A, B=B, x=x, y=y)

print('Equilibrium prices and quantity are',q_star,'and',p_star)

Equilibrium prices and quantity are 0.9509803921568627 and 98.09803921568627


In [24]:
ga = cobwebGA(n_individuals=n_individuals, n_generations=n_generations, n_bits=n_bits, q_bar=q_bar, cr=c_r, mr=m_r, A=A ,B=B , x=x , y=y)
outputs_df, all_outputs_df = ga.run(k=2, repair=True)

ga.graphs(variable='prices').save("../outputs/p_stable.png", dpi=600)
ga.graphs(variable='quantity').save("../outputs/q_stable.png", dpi=600)
ga.graphs(variable='SD_full').save("../outputs/DS_stable.png", dpi=600)

|████████████████████████████████████████| 100/100 [100%] in 12.3s (8.11/s)     


C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../outputs/p_stable.png
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../outputs/q_stable.png
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../outputs/DS_stable.png


No repair mechanism enabled

In [25]:
ga = cobwebGA(n_individuals=n_individuals, n_generations=n_generations, n_bits=n_bits, q_bar=q_bar, cr=c_r, mr=m_r, A=A ,B=B , x=x , y=y)
outputs_df, all_outputs_df = ga.run(k=2, repair=False)

ga.graphs(variable='prices').save("../outputs/p_stable_nr.png", dpi=600)
ga.graphs(variable='quantity').save("../outputs/q_stable_nr.png", dpi=600)
ga.graphs(variable='SD_full').save("../outputs/DS_stable_nr.png", dpi=600)

|████████████████████████████████████████| 100/100 [100%] in 11.1s (9.01/s)     


C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../outputs/p_stable_nr.png
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../outputs/q_stable_nr.png
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../outputs/DS_stable_nr.png


Unstable case

In [26]:
#demand parameters
A=2.296
B=0.0168
#cost function parameters
x=0
y=0.016

ga = cobwebGA(n_individuals=n_individuals, n_generations=n_generations, n_bits=n_bits, q_bar=q_bar, cr=c_r, mr=m_r, A=A ,B=B , x=x , y=y)
outputs_df, all_outputs_df = ga.run(k=2, repair=True)

ga.graphs(variable='prices').save("../outputs/p_unstable.png", dpi=600)
ga.graphs(variable='quantity').save("../outputs/q_unstable.png", dpi=600)
ga.graphs(variable='SD_full').save("../outputs/DS_unstable.png", dpi=600)

|████████████████████████████████████████| 100/100 [100%] in 12.3s (8.19/s)     


C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../outputs/p_unstable.png
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../outputs/q_unstable.png
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
C:\Users\iodio\AppData\Roaming\Python\Python310\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../outputs/DS_unstable.png
